# 3D Droplet Oscillation (Restart script)

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
var myBatch = ExecutionQueues[1];
//var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,FDY\smuda,DC2,<null>,Normal,True,[ \\hpccluster\hpccluster-scratch\smuda\ == ]


## Restart infos

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

43

In [ ]:
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.


In [ ]:
var workSess = wmg.Sessions.Skip(numSess); //.Take(12);
workSess

Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session ff159894-3ab0-4961-a743-3520554919c0 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\ff159894-3ab0-4961-a743-3520554919c0\Session.info'.'


#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_restart2	07/11/2022 13:39:12	95fa458f...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_restart2	07/11/2022 13:38:30	b97b80ef...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04*	07/11/2022 13:56:43	5480a5ac...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m4_Oh01_eta01*	07/11/2022 13:56:41	d98b11be...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015*	07/11/2022 13:49:48	53060394...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m4_Oh01_eta01_thirdOrderInit*	07/11/2022 13:49:03	325af26a...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015_thirdOrderInit*	07/11/2022 13:48:12	bc10dc66...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1*	07/11/2022 13:40:26	ea13d149...
#9: OscillatingDroplet3D	OD3D

In [ ]:
workSess.Pick(0)

OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...

In [ ]:
//workSess.Pick(42).Delete(true)

In [ ]:
int[] restartSessionIndex = new int[] { 0 };
restartSessionIndex.Length

1

In [ ]:
//var restartSessions = wmg.Sessions.Skip(18).Take(16);
var restartSessions = restartSessionIndex.Select(ind => workSess.ElementAt(ind));
restartSessions

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...


In [ ]:
List<(string sessionName, Guid restartID, string caseName)> cases = new List<(string sessionName, Guid restartID, string caseName)>();
foreach(var rSess in restartSessions) {
    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(rSess);
    var currSI = rSess;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }
    int restartNum = procSIs.Count;

    string orgName = rSess.Name;
    string rName;
    if (restartNum > 1) {
        rName = orgName.Substring(0, orgName.Length - 1) + restartNum;
    } else {
        rName = orgName + "_restart" + restartNum;
    }
    //Console.WriteLine("restart session name: {0}", rName);

    string[] subs = orgName.Split('_');
    string caseName = subs[4] + '_' + subs[5] + '_' + subs[6];
    //Console.WriteLine("restart case name: {0}", caseName);

    cases.Add((rName, rSess.ID, caseName));
}

In [ ]:
// List<(string sessionName, Guid restartID, string caseName)> cases = new List<(string sessionName, Guid restartID, string caseName)>();
// int nCase = 0;
// cases.Add((restartSessions.ElementAt(0).Name, restartSessions.ElementAt(0).ID, "m4_Oh056_eta005"));
// cases.Add((restartSessions.ElementAt(1).Name, restartSessions.ElementAt(1).ID, "m2_Oh01_eta02"));
// cases.Add((restartSessions.ElementAt(2).Name, restartSessions.ElementAt(2).ID, "m2_Oh01_eta01"));
// cases.Add((restartSessions.ElementAt(3).Name, restartSessions.ElementAt(3).ID, "m4_Oh01_eta01"));
// cases.Add((restartSessions.ElementAt(4).Name, restartSessions.ElementAt(4).ID, "m2_Oh01_eta04"));
// cases.Add((restartSessions.ElementAt(5).Name, restartSessions.ElementAt(5).ID, "m2_Oh01_eta01"));
// cases.Add((restartSessions.ElementAt(6).Name, restartSessions.ElementAt(6).ID, "m4_Oh01_eta01"));
// cases.Add((restartSessions.ElementAt(7).Name, restartSessions.ElementAt(7).ID, "m4_Oh056_eta005"));
// cases.Add((restartSessions.ElementAt(8).Name, restartSessions.ElementAt(8).ID, "m4_Oh056_eta005"));
// cases.Add((restartSessions.ElementAt(9).Name, restartSessions.ElementAt(9).ID, "m2_Oh01_eta02"));
// cases.Add((restartSessions.ElementAt(10).Name, restartSessions.ElementAt(10).ID, "m4_Oh01_eta04"));
// cases.Add((restartSessions.ElementAt(11).Name, restartSessions.ElementAt(11).ID, "m4_Oh056_eta005"));
// //cases.Add((restartSessions.ElementAt(12).Name, restartSessions.ElementAt(12).ID, "m4_Oh056_eta005"));

## Setup of control objects for all solver runs

In [ ]:
cases

index,Item1,Item2,Item3
0,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2,5e0fa717-0453-4cc2-a269-5dec740905d0,m2_Oh01_eta04


In [ ]:
// var case0 = cases.ElementAt(0);
// case0

Item1,Item2,Item3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReIni2,5e0fa717-0453-4cc2-a269-5dec740905d0,m2_Oh01_eta04


In [ ]:
// cases.Clear();
// cases.Add(("OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2", case0.restartID, case0.caseName));

In [ ]:
// int[] restartTimesteps = new int[cases.Count];
// restartTimesteps[0] = null;

In [ ]:
Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)> casesSetUp = new Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)>();

In [ ]:
casesSetUp.Add("m2_Oh01_eta04", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m2_Oh01_eta02", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m2_Oh01_eta01", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m3_Oh01_eta04", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m3_Oh01_eta03", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m3_Oh01_eta015", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m4_Oh01_eta04", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m4_Oh01_eta01", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m4_Oh056_eta005", (0.56, 1, 5e-3, 800));

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool bNewton = false;

foreach(int k in DegreeS) {
int iCase = 0;
foreach(var myCase in cases) {

    string JobName = $"{myCase.sessionName}";

    var C = new XNSE_Control();

    //C.ReInitPeriod = 10000;
    
    //C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    //C.InitialValues.Add("Phi", casesPhi[myCase]);
    C.RestartInfo = new Tuple<Guid, BoSSS.Foundation.IO.TimestepNumber>(myCase.restartID, null);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = casesSetUp[myCase.caseName].Ohnesorge;
    C.PhysicalParameters.mu_B = casesSetUp[myCase.caseName].Ohnesorge/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
        C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    }
    
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = casesSetUp[myCase.caseName].dt;
    C.NoOfTimesteps = casesSetUp[myCase.caseName].timesteps;
    C.Endtime = 7;
    
    int AMRlvl = casesSetUp[myCase.caseName].AMRlevel;
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";

    Console.WriteLine($"Case {iCase+1}: " + JobName);
    iCase++;
    
    Controls.Add(C);
    
}
}

Case 1: OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal for " + i);
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal for " + i + " and " + j);
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session ff159894-3ab0-4961-a743-3520554919c0 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\ff159894-3ab0-4961-a743-3520554919c0\Session.info'.'
Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\3190027a-c3d9-4869-bf85-ed028cbb97e2\Session.info'.'
Loading session ff159894-3ab0-4961-a743-3520554919c0 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\ff159894-3ab0-4961-a743-3520554919c0\Session.info'.'
Loading session 3190027a-c3d9-4869-bf85-ed028cbb97e2 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\Osc

In [ ]:
//wmg.AllJobs

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
//BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()